In [1]:
#import urllib, BeautifulSoup, pandas,time module and libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
from time import gmtime, strftime
import hashlib

def create_hash2():
    # setting the URL we want to monitor
    url = Request('http://125.19.52.218/realtime/', headers={'User-Agent': 'Mozilla/5.0'})

    #get the url and read it
    response = urlopen(url).read()
    currentHash = hashlib.sha224(response).hexdigest()
    
    #create a file, write it and close it
    file = open("hash2code.txt","w+")
    file.write(currentHash)
    file.close()

#calling create_hash function to generate hash code for the generated data
create_hash2()

#request and open the required url
url = urlopen("http://125.19.52.218/realtime/")

#make BeautifulSoup object
soup = BeautifulSoup(url, "html.parser")

#find the needed info using bs4 object
m1 = soup.find_all("h2")
m2 = soup.find_all("h3")
m3 = soup.find_all("div", {"class" : "tableData"})

#make variables and empty lists
i2 = 1
i3 = 2
master_list = []
master_list1 = []
master_list2 = []

#create list of place, city, type, date/time sample of water
for i1 in range(0,len(m1)):
    temp_dic = {}
    m6 = m1[i1].text.split("-")[1].split(",")
    if len(m6) == 2:
        temp_dic["Place"] = m6[0]
        temp_dic["City"] = m6[1]
    else:
        temp_dic["Place"] = m6[0]
    temp_dic["Type"] = m2[i2].text
    temp_dic["Last Updated"] = m2[i3].text
    i2 = i2 + 3
    i3 = i3 + 3
    master_list.append(temp_dic)
    
    
#create list of water sample other details
for element in m3:
    temp_dic={}
    m4=element.find_all("tr")
    for ele in m4:
        try:
            temp_dic[ele.find("td").text.strip()]=ele.find_all("td")[-1].text.strip()
        except:
            continue
    if temp_dic!={}:
        master_list1.append(temp_dic)

#adding lists to the DataFrame
df1 = pd.DataFrame(master_list)
df3 = pd.DataFrame(master_list1)

#mergeing the DataFrames
result = pd.concat([df1, df3], axis=1, join='inner')

In [2]:
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
import numpy as np
# declare an empty list to store lat/log
longitude = []
latitude = []
   
# function to find the coordinate of a state 
def findGeocode(state):
    #handling of exception occured by geolocator 
    try:  
        # Specify the user_agent as your
        # app name it should not be none
        geolocator = Nominatim(user_agent="your_app_name")
        return geolocator.geocode(state) 
    except GeocoderTimedOut:
          return findGeocode(state)    

#sending names of "STATE" to the function                    
for i in (result["Place"]):
    if findGeocode(i) != None:   
        loc = findGeocode(i)
          
        # coordinates returned from fun. added in list
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
       
    # if coordinate not find then add "NAN"
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)

# now add this column to dataframe
result["Longitude"] = longitude
result["Latitude"] = latitude

#creating the csv file of the scraped data
result.to_csv('WaterQualityData2_{}.csv'.format(strftime("%d %b %Y %H %M %S", gmtime())),index=True)

#displaying the generated csv file data
display(result)


,Place,City,Type,Last Updated,BOD(mg/l),DO(mg/l),Conductivity(µS/cm),pH,Temperature(°C),Ammonia(mg/l),...,Nitrate(mg/l),Potassium(mg/l),BTX(µg/ml),Water Level(cm.),Turbidity(NTU),TOC(mg/l),TOC(mg/l ),Color(µS/cm),Longitude,Latitude
0,Jagjeetpur STP drain,Haridwar,Drain,30-06-2021 06:04 PM,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,19.4,NaN,NaN,NaN,NaN,NaN,NaN
1,Madhya Ganga Barrage,Bijnore,River Ganga,27-07-2021 08:00 AM,0.74,1.93,250.0,7.89,26.9,0.99,...,0.87,NaN,NaN,250.0,5.24,4.11,NaN,NaN,NaN,NaN
2,Sukratal Ghat,Bijnore,River Banganga (Tributary),29-07-2021 09:01 AM,5.38,2.7,281.0,7.85,26.66,1.89,...,3.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Narora Barrage,Narora,River Ganga,29-07-2021 10:00 AM,1.86,7.72,171.0,7.67,23.8,1.05,...,0.41,NaN,NaN,178.04,NaN,NaN,9.38,NaN,NaN,NaN
4,Kachla Ghat Bridge,Badaun,River Ganga,29-07-2021 10:01 AM,4.17,7.33,194.54,7.86,25.2,1.25,...,0.12,NaN,0.01,449.86,713.41,12.81,NaN,NaN,NaN,NaN
5,Shahbad Bridge,Moradabad (D/s),River Ramganga (Tributary),29-07-2021 09:04 AM,NA,6.86,NA,8.08,30.0,1.24,...,NA,5.7,NaN,98.08,NA,NA,NaN,NaN,76.840747,30.148551
6,Khudaganj Bridge,Farrukhabad,River Kali (Tributary),28-07-2021 04:02 PM,5.24,5.06,225.0,7.49,30.3,2.6,...,1.41,8.8,0.04,350.0,280.81,6.25,NaN,NaN,79.713321,28.146514
7,Ghatiyaghat Bridge,Farrukhabad,River Ganga,29-07-2021 09:04 AM,2.44,NA,197.0,7.89,28.2,1.04,...,0.84,3.4,0.01,893.0,389.59,4.49,NaN,NaN,NaN,NaN
8,Manimau Bridge (Mehendi Ghat),Kannauj,River Ganga,29-07-2021 10:01 AM,2.92,6.56,212.0,8.21,29.4,0.15,...,2.3,3.8,0.02,417.3,133.44,4.83,NaN,NaN,NaN,NaN
9,Allahganj Bridge,Kannauj,River Ramganga (Tributary),29-07-2021 08:04 AM,3.16,3.68,246.0,7.98,27.9,0.87,...,3.18,5.0,0.09,278.88,206.87,NA,NaN,NaN,79.686607,27.545648
